In [11]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import pickle
from gensim.models import word2vec
import MeCab

In [12]:
# Read recipe inputs
raw_ramen = dataiku.Dataset("raw_ramen")
df = raw_ramen.get_dataframe()
df_ramen = df.groupby(['store_name','score','review_cnt'])['review'].apply(list).apply(' '.join).reset_index().sort_values('score', ascending=False)

w2v_folder = dataiku.Folder("m9JZdV7b").get_path()
text_folder = dataiku.Folder("aLTWBozg").get_path()
wakati_folder = dataiku.Folder("0kM5kXKs").get_path()
tagger_path = '-Owakati -d ' + wakati_folder
tagger_path

'-Owakati -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd'

In [14]:
df_ramen.review[0]

'どうも、Mormorです！今日はTwitter情報で、リニューアル前の「40番」のラーメン「洋」をイメージした限定ラーメン「洋」を提供するとの事で、「81番」に行って来ました。「40番」のラーメン「洋」は、"「5種類の野菜を丹念に煮込んだスープの"野菜出汁"。生のトマトの数倍のうま味を持つドライトマトを味のアクセントにした、唯一無二の洋を感じる洋風醤油ラーメン」"]と言うもので、ドライトマトの旨味を引き出した一杯でした。ドライトマトを使用したラーメンと言えば、「Japanese Soba Noodles 蔦」や「饗\u3000くろ喜」、そして「饗\u3000くろ喜」監修の「黄金拉麺\u3000鶏のおかげ」などでもローストトマトが使われています。今回の「洋」はプラス1種類の6種類の野菜を使用しているそうです。お店の入口横にも「限定!!!\u3000醤油ラーメン『洋』」と貼り出してあります。入口左手の券売機で食券を購入します。「洋」は一番右上の「限定」¥950の食券です。店員さんに食券を渡すと、細麺か手揉みの中太麺が選べるとの事で、確か「40番」の「洋」は細麺だったと思うので、今回は手揉みの中太麺にしました。待つ事7〜8分で「洋」が着丼。見た目は確かに「40番」のラーメン「洋」っぽいです。「40番」のラーメン「洋」をリスペクトした「81番」のラーメン「洋」でしたが、派手さは有りませんがしみじみ旨いタイプの一杯で、個人的にはとても好感が持てます。個人的に「40番」のラーメン「洋」はリニューアル前のラーメンの中では一番気に入っていて、限定とは言え、また食べられたのはとても嬉しいです。また、たまに出してくれるとありがたいです。詳細はこちら→https://wp.me/pb52jZ-4ZC 【2021.3.25再訪】町田まで買い物に出かけたこの日、せっかくなので昼飯を食べて帰ることに。以前は仕事で度々町田に訪れていましたが、最近は仕事のご縁が無くなったので、こうして買い物にでも出かけない限り滅多に町田に出かけなくなりました。食事する機会も当然減っていたのです。さて、どこで食べようかな…。この前の週に移転後初めて『町田汁場 進化 町田駅前店』で食べたばかり。なので今回は別の店で。特にラーメンに拘っていたわけではないのですが、仲見世通りを進んでいく途中でこの店に吸い込まれるように

In [0]:
tagger = MeCab.Tagger(tagger_path)#タグはMeCab.Tagger（neologd辞書）を使用
tagger.parse('')

In [0]:
def tokenize_ja(text, lower):
    node = tagger.parseToNode(str(text))
    while node:
        if lower and node.feature.split(',')[0] in ["名詞","形容詞"]:#分かち書きで取得する品詞を指定
            yield node.surface.lower()
        node = node.next
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]

In [0]:
#コーパス作成
wakati_ramen_text = []
for i in df_ramen['review']:
    txt = tokenize(i, 2, 10000, True)
    wakati_ramen_text.append(txt)
np.savetxt(text_folder + "/ramen_corpus.txt", wakati_ramen_text, fmt = '%s', delimiter = ',')

In [0]:
# モデル作成
word2vec_ramen_model = word2vec.Word2Vec(wakati_ramen_text, sg = 1, size = 100, window = 5, min_count = 5, iter = 100, workers = 3)
#sg（0: CBOW, 1: skip-gram）,size（ベクトルの次元数）,window（学習に使う前後の単語数）,min_count（n回未満登場する単語を破棄）,iter（トレーニング反復回数）

In [0]:
# モデルのセーブ
word2vec_ramen_model.save(w2v_folder+"/word2vec_ramen_model.model")

max_vocab = 30000 #40000にしても結果は同じだった
vocab = list(word2vec_ramen_model.wv.vocab.keys())[:max_vocab]
vectors = [word2vec_ramen_model.wv[word] for word in vocab]

vocab_df = pd.DataFrame(vectors)
vocab_df['words'] = vocab

py_recipe_output = dataiku.Dataset("ramen_vocab")
py_recipe_output.write_with_schema(vocab_df)